In [1]:
import cv2
import eos
import glob
import numpy as np

from face_alignment import FaceAlignment, LandmarksType
from preprocess import calibrate

In [2]:
np.set_printoptions(suppress=True)

In [3]:
images = [cv2.imread(f) for f in glob.glob('./data/*.jpg')]
images = [cv2.resize(img, (1920, 1080)) for img in images]

In [4]:
ret, mtx, dist, rvecs, tvecs = calibrate(images)

Camera calibrated successfully, re-projection error: 0.064171


In [5]:
mtx

array([[1477.45381478,    0.        , 1059.01536757],
       [   0.        , 1475.03559662,  569.81485806],
       [   0.        ,    0.        ,    1.        ]])

In [7]:
ret, frame = cv2.VideoCapture('./data/face.mp4').read()

In [8]:
fa3d = FaceAlignment(LandmarksType._3D,
                     flip_input=False,
                     device='cpu',
                     face_detector='blazeface')

In [53]:
[landmarks] = fa3d.get_landmarks(frame) 

___

In [17]:
model = eos.morphablemodel.load_model("eos/share/sfm_shape_3448.bin")

In [19]:
landmark_mapper = eos.core.LandmarkMapper('eos/share/ibug_to_sfm.txt')

In [18]:
shape_model = model.get_shape_model()

In [54]:
landmarks = landmarks[:, :2]

In [56]:
class PNP:
    
    ibug_ids_to_use = sorted([
        28, 29, 30, 31,  # nose ridge
        32, 33, 34, 35, 36,  # nose base
        37, 40,  # left-eye corners
        43, 46,  # right-eye corners
    ])
    
    def __init__(self):
        self.model = eos.morphablemodel.load_model('eos/share/sfm_shape_3448.bin')
        self.shape_model = self.model.get_shape_model()
        self.landmarks_mapper = eos.core.LandmarkMapper('eos/share/ibug_to_sfm.txt')
        self.sfm_points_ibug_subset = np.array([
            self.shape_model.get_mean_at_point(
                int(self.landmarks_mapper.convert(str(d)))
            )
            for d in range(1, 69)
            if self.landmarks_mapper.convert(str(d)) is not None
        ])
        self.sfm_points_for_pnp = np.array([
            self.shape_model.get_mean_at_point(
                int(self.landmarks_mapper.convert(str(d)))
            )
            for d in self.ibug_ids_to_use
        ])
        
    def solve(self, landmarks, camera_matrix):
        landmarks = np.array([
            landmarks[i - 1, :]
            for i in self.ibug_ids_to_use
        ], dtype=np.float64)
        success, rvec, tvec, inliers = cv2.solvePnPRansac(self.sfm_points_for_pnp, landmarks,
                                                          camera_matrix, None, flags=cv2.SOLVEPNP_EPNP)
        success, rvec, tvec = cv2.solvePnP(self.sfm_points_for_pnp, landmarks, camera_matrix, None,
                                           rvec=rvec, tvec=tvec, useExtrinsicGuess=True, flags=cv2.SOLVEPNP_ITERATIVE)
        return rvec, tvec

In [57]:
PNP().solve(landmarks, mtx)

(array([[-2.91352882],
        [ 0.07632486],
        [-0.45311698]]),
 array([[-305.46629741],
        [-148.48374817],
        [1087.90933709]]))

___

could also get the pose from the eos model fitting

In [ ]:
sfrm_points_ibug_subset = np.array(shape_model.get_mean_at_point(
    landmark_mapper.convert
))

In [10]:
blendshapes = eos.morphablemodel.load_blendshapes("eos/share/expression_blendshapes_3448.bin")
morphablemodel_with_expressions = eos.morphablemodel.MorphableModel(
    model.get_shape_model(),
    blendshapes,
    color_model=eos.morphablemodel.PcaModel(),
    vertex_definitions=None,
    texture_coordinates=model.get_texture_coordinates()
)

In [11]:
image_height, image_width, _ = frame.shape

In [12]:
_landmarks = [
    eos.core.Landmark(str(i), [x, y]) 
    for i, [x, y] in enumerate(zip(landmarks[:, 0], landmarks[:, 1]))
]

In [13]:
edge_topology = eos.morphablemodel.load_edge_topology('eos/share/sfm_3448_edge_topology.json')
contour_landmarks = eos.fitting.ContourLandmarks.load('eos/share/ibug_to_sfm.txt')
model_contour = eos.fitting.ModelContour.load('eos/share/sfm_model_contours.json')

(mesh, pose, shape_coeffs, blendshape_coeffs) = eos.fitting.fit_shape_and_pose(morphablemodel_with_expressions,
    _landmarks, landmark_mapper, image_width, image_height, edge_topology, contour_landmarks, model_contour)


In [14]:
pose.get_modelview()

array([[  0.8888019 ,   0.02127027,   0.45779783, 518.33057   ],
       [ -0.06249542,   0.9952162 ,   0.07509327, 278.39148   ],
       [ -0.45401055,  -0.09535331,   0.88587934,   0.        ],
       [  0.        ,   0.        ,   0.        ,   1.        ]],
      dtype=float32)

In [15]:
pose.get_rotation()

array([-0.04389282,  0.23480578, -0.02157105,  0.97081125], dtype=float32)

In [16]:
pose.get_rotation_euler_angles()

array([-0.10722407,  0.47126138, -0.07019868], dtype=float32)